# 라이브러리 및 데이터 불러오기

In [0]:
!apt-get update -qq
!apt-get install -y fonts-nanum

In [0]:
from pyspark.sql.functions import when, col, to_date
from pyspark.sql import functions as F
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
import math
import re

In [0]:
# from matplotlib import font_manager as fm

# # 한글 글꼴 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'  # 윈도우에서 일반적으로 지원되는 한글 글꼴 (Mac의 경우 'AppleGothic' 등 사용)
# plt.rcParams['font.family'] = 'NanumGothic'  # 또는 'NanumGothic'
# plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호가 깨지는 문제 해결
# font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf")
# font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
# font_prop = fm.FontProperties(fname=font_path)


# font_dirs = ["/usr/share/fonts/truetype/nanum/"]
# # font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"

# font_prop = fm.FontProperties(fname=font_dirs)
 
# plt.rc('font', family='NanumGothic')
# plt.rc('axes', unicode_minus=False)
 
# pd.Series([-1,2,3]).plot(title='테스트', figsize=(3,2))
# pass

# 0. 공통

## 1) 유의미한 샘플 수

In [0]:
# # 전체 레코드 수 = raw는 항상 "18000000"
# total_count = amount_period_df.count()
# total_count

In [0]:
# 통계적으로 유의한 샘플 크기 계산
def calculate_sample_size(population_size, confidence_level=0.95, margin_error=0.05):
    """
    통계적으로 유의한 샘플 크기 계산
    """
    z_score = 2.576  # 99% 신뢰도
    p = 0.5  # 최대 분산
    
    n = (z_score**2 * p * (1-p)) / (margin_error**2)
    n_adjusted = n / (1 + (n-1)/population_size)
    
    return int(n_adjusted)

In [0]:
total_count = 18000000
print(f"전체 데이터: {total_count:,}")


# 통계적 샘플 크기 계산
sample_size = calculate_sample_size(total_count)
sample_fraction = sample_size / total_count

print(f"필요 샘플 크기: {sample_size:,}")
print(f"샘플링 비율: {sample_fraction:.7f}")

# 1. 기간 포함 컬럼

In [0]:
# 112개 컬럼
amount_period_df = spark.read.table("database_03_cache.amount_period_df")

In [0]:
# 112개 컬럼
display(amount_period_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 수치형 컬럼
period_numeric_cols = [c for c, t in amount_period_df.dtypes if t in ("int", "bigint", "float", "double")]

len(period_numeric_cols)

In [0]:
# 전체 레코드 수
total_count = amount_period_df.count()
total_count

## 1) 희소(sparse)한 컬럼 처리

In [0]:
# 최종 목표 
# 112(기존 컬럼 개수) - 8 (0이 100%인 컬럼 수) + 1 (0이 0.7%이상, 1미만인 컬럼을 이진화 후 컬럼 하나로 압축)
112 - 8 + 1 - 68

In [0]:
from pyspark.sql.functions import col, when, sum as Fsum
from pyspark.ml.feature import VectorAssembler

# 1. 전체 수치형 컬럼 중 0 비율 기준으로 나누기
period_zero_only_cols = []      # 전부 0인 컬럼
period_zero_ratio_cols = []     # 0이 70% 이상, 100% 미만인 컬럼


# 전부 0인 컬럼 탐지
sum_df = amount_period_df.select([Fsum(col(c)).alias(c) for c in period_numeric_cols])
for c in sum_df.columns:
    if sum_df.select(c).collect()[0][0] == 0:
        period_zero_only_cols.append(c)
print(len(period_zero_only_cols))

# 일부만 0인 희소 컬럼 탐지
for c in period_numeric_cols:
    if c not in period_zero_only_cols:
        zero_count = amount_period_df.filter(col(c) == 0).count()
        ratio = zero_count / total_count
        if 0.7 <= ratio < 1.0:
            period_zero_ratio_cols.append(c)
print(len(period_zero_ratio_cols))

In [0]:
import matplotlib.pyplot as plt

# 데이터
num_zero_only = 8
num_zero_ratio = 68     #112 - 8 + 1 - 68 = 45
num_total = 112
num_remaining = num_total - num_zero_only - num_zero_ratio

labels = ['0이 100%인 컬럼', '0이 70~99%인 컬럼', '그 외 컬럼']
sizes = [num_zero_only, num_zero_ratio, num_remaining]
colors = ['#15A2DB', '#065B9C', '#999999']

# 시각화
plt.figure(figsize=(6, 6))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=140,
    colors=colors
)

# 내부 텍스트(비율 텍스트) 스타일 조정
for autotext in autotexts:
    autotext.set_fontsize(15)
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 외부 라벨 텍스트 스타일 조정
for text in texts:
    text.set_fontsize(13)
    text.set_color('black')

plt.title('0 비율 기준 컬럼 구성 비율', fontsize=15)
plt.tight_layout()
plt.show()


In [0]:
# 2. 전부 0인 컬럼 제거
ap_df = amount_period_df.drop(*period_zero_only_cols)
print(len(ap_df.columns))  # 112-8 = 104

# 3. 이진화된 컬럼 추가
for zcol in period_zero_ratio_cols:
    ap_df = ap_df.withColumn(f"{zcol}_사용여부", when(col(zcol) > 0, 1).otherwise(0))
print(len(ap_df.columns))  # 104 + 68 = 172

# 4. 벡터화
assembler = VectorAssembler(
    inputCols=[f"{z}_사용여부" for z in period_zero_ratio_cols],
    outputCol="사용패턴벡터"
)
ap_df = assembler.transform(ap_df)
print(len(ap_df.columns))

# 5. 이진화된 컬럼 제거하고 벡터만 남김
ap_df = ap_df.drop(*[f"{z}_사용여부" for z in period_zero_ratio_cols])

# 6. 0이 70% 이상인 기존 컬럼도 삭제
ap_df = ap_df.drop(*period_zero_ratio_cols)


print(len(ap_df.columns))  #112 - 8 + 1 - 68 = 45

In [0]:
display(ap_df)

Databricks data profile. Run in Databricks to view.

## 2) 이상치 처리/스케일링 - 로그 변환
테이블 분석에서 box plot 확인 결과 대부분 positive skew로 이상치 많은 분포임. 따라서 로그 변환

In [0]:
print(len(ap_df.columns))

In [0]:
# 수치형 컬럼 - 컬럼 바뀌었으므로, 업데이트
ap_numeric_cols = [c for c, t in ap_df.dtypes if t in ("int", "bigint", "float", "double")]

len(ap_numeric_cols)

주의!! 그냥 하면, 음수값이 null 처리됨

In [0]:
# from pyspark.sql.functions import log1p, col
# # 로그 변환
# for col_name in ap_numeric_cols:
#     ap_df = ap_df.withColumn(col_name, log1p(col(col_name)))

from pyspark.sql.functions import log1p, col, when

# 로그 변환 (음수값은 0으로 대체 후 log1p 적용)
for col_name in ap_numeric_cols:
    ap_log_df = ap_df.withColumn(
        col_name,
        log1p(when(col(col_name) < 0, 0).otherwise(col(col_name)))
    )

In [0]:
display(ap_log_df)

Databricks data profile. Run in Databricks to view.


## 3) 상관관계 분석

- 전체 데이터: 18,000,000
- 필요 샘플 크기 (sample_size): 663
- 샘플링 비율 (sample_fraction): 0.0000368

#### 상관관계 함수 - 전체

In [0]:
%run /Workspace/Shared/utils

In [0]:
ap_corr_matrix, ap_corr_array, ap_cols_names = correlation_func(ap_log_df)

In [0]:
import numpy as np
import pandas as pd

def get_correlated_groups(corr_matrix, col_names, threshold=0.9):
    n = len(col_names)
    to_remove = set()
    groups = []

    visited = set()
    for i in range(n):
        if col_names[i] in visited:
            continue
        group = set()
        for j in range(i + 1, n):
            if abs(corr_matrix[i, j]) > threshold:
                group.add(col_names[i])
                group.add(col_names[j])
        if group:
            groups.append(group)
            visited.update(group)

    return groups

In [0]:
groups = get_correlated_groups(ap_corr_array, ap_cols_names, threshold=0.9)
for idx, g in enumerate(groups):
    print(f"[Group {idx+1}] {g}")

In [0]:
groups

In [0]:
!apt-get -y install fonts-nanum
import matplotlib.font_manager as fm
plt.rcParams['font.family'] = 'NanumGothic'


In [0]:
# NanumGothic (Linux 환경 기준)
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"  # 경로는 환경에 따라 수정
font_prop = fm.FontProperties(fname=font_path)
plt.rc('font', family= font_prop.get_name())
plt.rcParams['axes.unicode_minus'] = False

In [0]:
import networkx as nx
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

def visualize_highly_correlated_columns(corr_array, col_names, threshold=0.9, figsize=(16, 12)):
    """
    상관계수 행렬과 컬럼 이름 리스트를 받아 상관계수 기반 네트워크 그래프 시각화.
    
    """
    G = nx.Graph()
    n = len(col_names)

    # 엣지 생성
    for i in range(n):
        for j in range(i + 1, n):
            if abs(corr_array[i, j]) >= threshold:
                G.add_edge(col_names[i], col_names[j])

    # 라벨 줄바꿈 함수
    def format_label(label):
        parts = label.split('_')
        if len(parts) >= 3:
            return f"{parts[0]}_{parts[1]}\n{parts[-1]}"
        return label

    labels = {node: format_label(node) for node in G.nodes}

    # 시각화
    plt.figure(figsize=figsize)
    pos = nx.spring_layout(G, seed=42, k=0.6)

    nx.draw_networkx_nodes(G, pos, node_color="#065B9C", node_size=2200)
    nx.draw_networkx_edges(G, pos, edge_color="#AAAAAA", width=2)
    nx.draw_networkx_labels(G, pos, labels=labels, font_color="white",
                                                    font_family=font_prop.get_name(),  # 핵심!
                                                    font_size=11)
    
    plt.title("상관계수 {:.2f} 이상 컬럼 네트워크 시각화".format(threshold), fontsize=16, fontproperties=font_prop.get_name())
    plt.axis("off")
    plt.tight_layout()
    plt.show()

# 예시 실행
visualize_highly_correlated_columns(ap_corr_matrix, ap_cols_names, threshold=0.9)


In [0]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

# 상관계수가 0.9 이상인 컬럼쌍으로 엣지 구성
threshold = 0.9
edges = []
for i in range(len(ap_cols_names)):
    for j in range(i + 1, len(ap_cols_names)):
        if abs(ap_corr_matrix[i, j]) >= threshold:
            edges.append((ap_cols_names[i], ap_cols_names[j]))

# 네트워크 그래프 생성
G = nx.Graph()
G.add_edges_from(edges)

plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42, k=1.5)  # k 값 키우면 노드 간 거리 넓어짐
pos = nx.spring_layout(G, seed=42)  # 노드 위치 고정
nx.draw(G, pos, with_labels=True, node_color="#065B9C", font_color="white",
        node_size=1500, edge_color="#999999", width=2, font_size=12)
plt.title("상관계수 0.9 이상인 컬럼 간 관계 그래프")
plt.tight_layout()
plt.show()



상관관계 - 특정 컬럼 포함된 경우 출력

In [0]:
# 특정 컬럼 포함된 경우 찾기
def find_high_corr_pairs_for_feature(high_corrs_list, feature_name):
    """
    특정 feature가 포함된 높은 상관관계 쌍만 필터링하여 출력

    Parameters:
        high_corrs_list (List[Dict]): get_high_correlations() 함수 결과 리스트
        feature_name (str): 검색할 피처 이름

    Returns:
        List[Dict]: 해당 feature가 포함된 상관쌍 리스트
    """
    related_corrs = [pair for pair in high_corrs_list 
                     if feature_name in [pair['feature1'], pair['feature2']]]

    if not related_corrs:
        print(f"'{feature_name}'은(는) 높은 상관관계 쌍에 포함되지 않았습니다.")
    else:
        print(f"\n'{feature_name}' 관련 높은 상관관계 쌍 ({len(related_corrs)}개):")
        for pair in related_corrs:
            print(f"{pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")
    
    return related_corrs


In [0]:
# find_high_corr_pairs_for_feature(ap_high_corrs, '이용금액_온라인_R3M')

In [0]:
plot_correlation_heatmaps(corr_array=ap_corr_array,
                          feature_names=ap_cols_names, threshold=0.9)


In [0]:
plot_correlation_heatmaps(corr_array=ap_corr_array,
                          feature_names=ap_cols_names)


### 3-1) 기간별 상관관계 분석

- **층화 샘플링 함수 : stratified_sampling** <br>
    `ap_sampled_df = stratified_sampling(ap_df, sample_fraction)`
- **상관관계 분석 함수 (피쳐 수 제한 없음) : fast_correlation_analysis** <br>
    `    correlation_matrix, feature_names = fast_correlation_analysis(period_df, period_columns)`
- **높은 상관관계 분석 함수 : get_high_correlations** <br>
    `ap_high_corrs, ap_corr_array = get_high_correlations(ap_corr_matrix, ap_cols_names)`
- **전체 및 높은 상관관계 시각화 : plot_correlation_heatmaps** <br>
    `plot_correlation_heatmaps(corr_array=ap_corr_array,
                          feature_names=ap_cols_names)`
- **다중공선성 검사 함수 : check_multicollinearity** <br>
    `ap_multicollinear = check_multicollinearity(ap_corr_matrix, ap_cols_names, 0.9)`

In [0]:
### 기간별 메인 분석 함수
def analyze_correlation_by_period(period_columns, period_name, full_df):
    """
    특정 기간의 컬럼들에 대해 상관관계 분석을 수행하는 함수들을 한 데 모은 함수
    (stratified_sampling, fast_correlation_analysis, get_high_correlations, check_multicollinearity)
    """
    print(f"\n{'='*10}")
    print(f"=== {period_name} 기간 상관관계 분석 ===")
    print(f"{'='*10}")
    print(f"분석 대상 컬럼 수: {len(period_columns)}")

    # 1. 층화 샘플링
    sampled_df = stratified_sampling(full_df, sample_fraction)
    print(f"샘플 데이터: {sampled_df.count():,}")

    # 2. 기간 컬럼만 선택
    period_df = sampled_df.select(period_columns)

    # 3. 상관관계 분석
    correlation_matrix, feature_names = fast_correlation_analysis(period_df, period_columns)
    print("상관관계 계산 완료!")

    # 4. 높은 상관관계 추출 및 출력
    high_corrs = high_correlations(correlation_matrix, feature_names)
    print(f"\n==높은 상관관계(|r| > 0.7) : {len(high_corrs)}개")

    return correlation_matrix, feature_names

In [0]:
# ✔ 사용된 기간들: ['B0M', 'B1M', 'B2M', 'R3M', 'R6M', 'R12M']

In [0]:
import re
from collections import defaultdict
import pandas as pd

# 1. 기간 목록 정의
period_list = ['B0M', 'B1M', 'B2M', 'R3M', 'R6M', 'R12M']

# 2. 기간별 컬럼 분류 함수 리팩토링
def classify_columns_by_period(df_columns):
    """
    컬럼명을 주어진 period_list에 따라 분류하는 함수
    """
    period_groups = {period: [] for period in period_list}
    
    for col_name in df_columns:
        # 제외할 컬럼들
        if col_name in ['기준년월', '발급회원번호']:
            continue
            
        # 각 기간 패턴에 매칭되는지 확인
        for period in period_list:
            if col_name.endswith(f"_{period}"):
                period_groups[period].append(col_name)
                break
    
    return period_groups

In [0]:
# 함수 실행
classified_columns = classify_columns_by_period(ap_df.columns)
display(classified_columns)

# 결과 확인
classified_df = pd.DataFrame([(k, len(v), v[:3]) for k, v in               
                              classified_columns.items()],
                              columns=["기간", "컬럼 수", "예시 컬럼(최대 3개)"])
classified_df

In [0]:
correlation_results = {}

# period_list = ['B0M', 'B1M', 'B2M', 'R3M', 'R6M', 'R12M']
for period, cols in classified_columns.items():
    if period in period_list:
        corr_matrix, feature_names = analyze_correlation_by_period(cols, period, ap_df)
        if corr_matrix is not None:
            correlation_results[period] = {'matrix': corr_matrix, 'features': feature_names}


- 전체_다중공선성 위험 : 59개
- 각각_다중공선성 위험 : 5 + 0 + 0 + 5 + 4 + 25 = 39개 <br>
⇒ 기간 외 제거해야 하는 것 : 20개

In [0]:
# correlation_results

In [0]:
multicollinear_all = []

for period, result in correlation_results.items():
    matrix = result["matrix"]
    features = result["features"]
    corr_array = matrix.toArray()
    
    for i in range(len(features)):
        for j in range(i+1, len(features)):
            corr_value = corr_array[i][j]
            if abs(corr_value) > 0.9:
                multicollinear_all.append({
                    'feature1': features[i],
                    'feature2': features[j],
                    'correlation': corr_value,
                    'period': period
                })

print(f"총 다중공선성 쌍: {len(multicollinear_all)}")
multicollinear_all

In [0]:
# 중복 컬럼 포함된 쌍

from collections import Counter

flat = [item for pair in multicollinear_all for item in [pair["feature1"], pair["feature2"]]]
Counter(flat).most_common()


### 3-2) 상관계수 기반_ 컬럼 처리

#### 기간 내 다중공선성 쌍 필터링

In [0]:
import re
from collections import defaultdict

# 우선순위 정의
priority_order = ["신용", "신판", "일시불", "최대이용금액"]

# 동일 기간 내 다중공선성 제거 대상 추출
def extract_period(colname):
    """컬럼명에서 기간 추출"""
    match = re.search(r'_R\d+M|_B\d+M', colname)
    return match.group() if match else ''

def extract_type(colname):
    """컬럼명에서 타입(신용/신판/일시불 등) 추출"""
    for p in priority_order:
        if p in colname:
            return p
    return colname  # 매칭 안 될 경우 그대로 반환

In [0]:
# 제거 대상
remove_candidates = set()
grouped_by_period = defaultdict(list)

# 1. 기간별로 쌍 그룹화
for pair in multicollinear_all:
    grouped_by_period[pair["period"]].append((pair["feature1"], pair["feature2"]))

# 2. 우선순위 기반 제거
for period, pairs in grouped_by_period.items():
    for f1, f2 in pairs:
        t1, t2 = extract_type(f1), extract_type(f2)
        
        if t1 in priority_order and t2 in priority_order:
            if priority_order.index(t1) < priority_order.index(t2):
                remove_candidates.add(f2)
            else:
                remove_candidates.add(f1)
        elif t1 in priority_order:
            remove_candidates.add(f2)
        elif t2 in priority_order:
            remove_candidates.add(f1)
        else:
            remove_candidates.add(f2)  # 임의 제거

remove_candidates = sorted(remove_candidates)
print(f"제거할 컬럼 수: {len(remove_candidates)}")
for col in remove_candidates:
    print(" -", col)


In [0]:
# 검증: 제거 대상 컬럼이 각 쌍에서 적어도 하나는 포함되었는지?
unresolved_pairs = []

for pair in multicollinear_all:
    if pair["feature1"] not in remove_candidates and pair["feature2"] not in remove_candidates:
        unresolved_pairs.append(pair)

print(f"⚠️ 제거되지 않은 위험한 쌍 수: {len(unresolved_pairs)}")

**제거**

In [0]:
# 다중공선성 제거된 최종 데이터
ap_reduced_df = ap_df.drop(*remove_candidates)

print(f"원본 컬럼 수: {len(ap_df.columns)}")
print(f"최종 컬럼 수: {len(ap_reduced_df.columns)}")


In [0]:
105-27

In [0]:
display(ap_reduced_df)

Databricks data profile. Run in Databricks to view.

#### 기간 외 다중공선성 쌍 필터링

In [0]:
from pyspark.sql.functions import col

# 층화 샘플링 실행
print("=== 층화 샘플링 실행 ===")
apr_sampled_df = stratified_sampling(ap_reduced_df, sample_fraction)
apr_sampled_count = apr_sampled_df.count()
print(f"샘플 데이터: {apr_sampled_count:,}")

# 빠른 분석 실행 (모든 피처 사용)
apr_corr_matrix, apr_cols_names = fast_correlation_analysis(apr_sampled_df)
print("상관관계 계산 완료!")

# 높은 상관관계 분석
apr_high_corrs, apr_corr_array = get_high_correlations(apr_corr_matrix, apr_cols_names)

apr_high_corrs

In [0]:
# 다중공선성 검사
apr_multicollinear = check_multicollinearity(apr_corr_matrix, apr_cols_names, 0.9)

print(f"\n=== 다중공선성 위험 ({len(apr_multicollinear)}개) ===")
for pair in apr_multicollinear:
    print(f"⚠️ {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")

⇒ 모두 동일 컬럼 내용에, 기간만 다른 경우임
- R12M 기준으로 대표 기간 컬럼 유지
  - → 장기 소비 패턴 기반으로 고객군 구분에 적합

In [0]:
apr_multicollinear

In [0]:
import re

# 우선순위 매핑
period_priority = ['R12M', 'R6M', 'R3M', 'B0M', 'B1M', 'B2M']
period_rank = {p: i for i, p in enumerate(period_priority)}

# 제거 대상 저장
drop_cols = set()

# 각 공선성 쌍에 대해 비교
for pair in apr_multicollinear:
    f1, f2 = pair["feature1"], pair["feature2"]

    # 기간 추출
    m1 = re.search(r'(B\d+M|R\d+M)', f1)
    m2 = re.search(r'(B\d+M|R\d+M)', f2)

    # 둘 다 기간을 가진 경우만 처리
    if m1 and m2:
        p1, p2 = m1.group(), m2.group()

        # 우선순위 비교
        if period_rank.get(p1, 999) < period_rank.get(p2, 999):
            drop_cols.add(f2)
        else:
            drop_cols.add(f1)

# 결과 확인
print(f"🔍 공선성 기반 제거 대상 수: {len(drop_cols)}")
print(f"🧹 제거할 컬럼 리스트:\n{sorted(drop_cols)}")


In [0]:
drop_cols

In [0]:
# 제거 실행
ap_rr_df = ap_reduced_df.drop(*drop_cols)

print(f"최종 컬럼 수 (클러스터링용): {len(ap_rr_df.columns)}")


### 💡 데이터 추출

In [0]:
### 데이터 베이스 사용 설정
ap_rr_df = ap_rr_df.cache()
spark.sql("USE database_pjt")
print("현재 데이터베이스를 'database_pjt'로 설정")

ap_rr_df.write.mode("overwrite").saveAsTable("3_amount_period")
print("이용금액(기간 포함) 관련 테이블 생성 완료")

In [0]:
### 데이터 베이스 사용 설정
ap_rr_df = ap_rr_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

ap_rr_df.write.mode("overwrite").saveAsTable("amount_period_df_fin")
print("이용금액(기간 포함) 관련 테이블 생성 완료")

In [0]:
### 확인
df1 = spark.read.table("database_03_cache.amount_period_df_fin")

print(len(df1.columns))
df1.printSchema()

---

# 2. 기간 불포함 컬럼

In [0]:
amount_nonperiod_df = spark.read.table("database_03_cache.amount_nonperiod_df")

In [0]:
amount_nonperiod_df = amount_nonperiod_df.drop('이용금액대')

In [0]:
amount_nonperiod_columns = amount_nonperiod_df.columns
len(amount_nonperiod_columns)

In [0]:
# 65개 컬럼
display(amount_nonperiod_df)

Databricks data profile. Run in Databricks to view.

In [0]:
# 수치형 컬럼
nonperiod_numeric_cols = [c for c, t in amount_nonperiod_df.dtypes if t in ("int", "bigint", "float", "double")]

len(nonperiod_numeric_cols)

In [0]:
# 전체 레코드 수
total_count = amount_nonperiod_df.count()
total_count

## 1) 희소(sparse)한 컬럼 처리_

In [0]:
# 최종 목표 
# 64(기존 컬럼 개수) - 3 (0이 100%인 컬럼 수) + 1 (0이 0.7%이상, 1미만인 컬럼을 이진화 후 컬럼 하나로 압축)
64 -3 + 1 - 37     #25

In [0]:
from pyspark.sql.functions import col, when, sum as Fsum
from pyspark.ml.feature import VectorAssembler

# 1. 전체 수치형 컬럼 중 0 비율 기준으로 나누기
zero_ratio_cols = []     # 0이 70% 이상, 100% 미만인 컬럼
zero_only_cols = []      # 전부 0인 컬럼


# 전부 0인 컬럼 탐지
sum_df = amount_nonperiod_df.select([Fsum(col(c)).alias(c) for c in nonperiod_numeric_cols])
for c in sum_df.columns:
    if sum_df.select(c).collect()[0][0] == 0:
        zero_only_cols.append(c)
print(len(zero_only_cols))

# 일부만 0인 희소 컬럼 탐지
for c in nonperiod_numeric_cols:
    if c not in zero_only_cols:
        zero_count = amount_nonperiod_df.filter(col(c) == 0).count()
        ratio = zero_count / total_count
        if 0.7 <= ratio < 1.0:
            zero_ratio_cols.append(c)
print(len(zero_ratio_cols))

In [0]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

# 사용자 코드 실행 이후 결과값 예시 (실제 값은 사용자 환경에서 가져와야 함)
num_zero_only = len(zero_ratio_cols)
num_zero_ratio = len(zero_only_cols)
num_total = len(amount_nonperiod_columns)

num_remaining = num_total - num_zero_only - num_zero_ratio

# 라벨과 값
labels = ['0이 100%인 컬럼', '0이 70~99%인 컬럼', '그 외 컬럼']
sizes = [num_zero_only, num_zero_ratio, num_remaining]
colors = ['#15A2DB', '#065B9C', '#999999']

# 시각화
plt.figure(figsize=(6, 6))
wedges, texts, autotexts = plt.pie(
    sizes,
    labels=labels,
    autopct='%1.1f%%',
    startangle=140,
    colors=colors
)

# 내부 텍스트(비율 텍스트) 스타일 조정
for autotext in autotexts:
    autotext.set_fontsize(15)
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 외부 라벨 텍스트 스타일 조정
for text in texts:
    text.set_fontsize(13)
    text.set_color('black')

plt.title('0 비율 기준 컬럼 구성 비율', fontsize=15)
plt.tight_layout()
plt.show()

In [0]:
# 2. 전부 0인 컬럼 제거
anp_df = amount_nonperiod_df.drop(*zero_only_cols)
print(len(anp_df.columns))  # 64-3 =61

# 3. 이진화된 컬럼 추가
for zcol in zero_ratio_cols:
    anp_df = anp_df.withColumn(f"{zcol}_사용여부", when(col(zcol) > 0, 1).otherwise(0))
print(len(anp_df.columns))  # 61 + 37 = 98

# 4. 벡터화
assembler = VectorAssembler(
    inputCols=[f"{z}_사용여부" for z in zero_ratio_cols],
    outputCol="사용패턴벡터"    # 98+1 = 99
)
anp_df = assembler.transform(anp_df)
print(len(anp_df.columns))

# 5. 이진화된 컬럼 제거하고 벡터만 남김
anp_df = anp_df.drop(*[f"{z}_사용여부" for z in zero_ratio_cols])
print(len(anp_df.columns))   # 99-37 = 62

# 6. 0이 70% 이상인 기존 컬럼도 삭제
anp_df = anp_df.drop(*zero_ratio_cols)

print(len(ap_df.columns))  # 64 -3 + 1 - 37     #25
display(ap_df)

In [0]:
display(anp_df)

Databricks data profile. Run in Databricks to view.

## 2) 이상치 처리/스케일링 - 로그 변환
테이블 분석에서 box plot 확인 결과 대부분 positive skew로 이상치 많은 분포임. 따라서 로그 변환

In [0]:
# 수치형 컬럼
anp_numeric_cols = [c for c, t in anp_df.dtypes if t in ("int", "bigint", "float", "double")]

len(anp_numeric_cols)
anp_numeric_cols

In [0]:
from pyspark.sql.functions import log1p, col
# anp = anp_df.copy()

# 로그 변환
for col_name in anp_numeric_cols:
    anp_df = anp_df.withColumn(col_name, log1p(col(col_name)))
# 음수 없는 듯

In [0]:
display(anp_df)

Databricks data profile. Run in Databricks to view.

## 3) 상관관계 분석 -
pyspark.ml.stat.Correlation은 **벡터 열**(아래 코드에서 features변수)에에서만 작동하므로<br>
→ 반드시 VectorAssembler 사용해야 함

In [0]:
# 1. 층화 샘플링 실행
print("=== 층화 샘플링 실행 ===")
anp_sampled_df = stratified_sampling(anp_df, sample_fraction)
anp_sampled_count = anp_sampled_df.count()
print(f"샘플 데이터: {anp_sampled_count:,}")


In [0]:
# 2. 빠른 분석 실행 (모든 피처 사용)
anp_corr_matrix, anp_cols_names = fast_correlation_analysis(anp_sampled_df)
print("상관관계 계산 완료!")

In [0]:
# 3. 높은 상관관계 분석
anp_high_corrs, anp_corr_array = get_high_correlations(anp_corr_matrix, anp_cols_names)

# anp_high_corrs

- 이용금액_교통 ↔ 교통_전체이용금액: 1.000  
- 이용금액_교통 ↔ _1순위교통업종_이용금액: 0.967  
- 교통_전체이용금액 ↔ _1순위교통업종_이용금액: 0.967  
- 교통_주유이용금액 ↔ _1순위교통업종_이용금액: 0.951
- 이용금액_교통 ↔ 교통_주유이용금액: 0.916  
- 교통_주유이용금액 ↔ 교통_전체이용금액: 0.916
- 이용금액_교통 ↔ _3순위업종_이용금액: 0.821
- 교통_전체이용금액 ↔ _3순위업종_이용금액: 0.821
- 이용금액_교통 ↔ _2순위교통업종_이용금액: 0.812
- 교통_전체이용금액 ↔ _2순위교통업종_이용금액: 0.812
- _3순위업종_이용금액 ↔ _1순위교통업종_이용금액: 0.777
- 교통_주유이용금액 ↔ _3순위업종_이용금액: 0.724
- _2순위교통업종_이용금액 ↔ _3순위교통업종_이용금액: 0.720
- 이용금액_교통 ↔ 이용금액_사교활동: 0.711
- 이용금액_사교활동 ↔ 교통_전체이용금액: 0.711
- 이용금액_교통 ↔ _2순위업종_이용금액: 0.705
- 교통_전체이용금액 ↔ _2순위업종_이용금액: 0.705

In [0]:
# 4. 다중공선성 검사 (0.9 이상)
anp_multicollinear = check_multicollinearity(anp_corr_matrix, anp_cols_names, 0.9)

print(f"\n=== 다중공선성 위험 ({len(anp_multicollinear)}개) ===")
for pair in anp_multicollinear:
    print(f"⚠️ {pair['feature1']} ↔ {pair['feature2']}: {pair['correlation']:.3f}")

In [0]:
# 5. 시각화
plot_correlation_heatmaps(corr_array = anp_corr_array,
                          feature_names = anp_cols_names)


### 3-1) 상관계수 기반_ 컬럼 처리

In [0]:
drop_nonperiod_cols = [
    '이용금액_쇼핑',
    '_1순위업종_이용금액',

    '이용금액_교통',
    '교통_주유이용금액',
    '_1순위교통업종_이용금액',

    '이용금액_납부',
    '_1순위납부업종_이용금액',

    '이용금액_여유생활',
    '여유_전체이용금액',
    '_1순위여유업종_이용금액',

    '쇼핑_온라인_이용금액',
    '_3순위쇼핑업종_이용금액'
]
len(drop_nonperiod_cols)


In [0]:
# rename_dict = {
#     "교통_주유이용금액": "교통_주유이용금액(1순위교통_전체)",
#     "이용후경과월_일시불": "일시불_경과월",
#     "이용후경과월_할부": "할부_경과월"
# }

# for old_col, new_col in rename_dict.items():
#     df = df.withColumnRenamed(old_col, new_col)


In [0]:
from collections import Counter

# 각 쌍에서 등장한 모든 변수 리스트 생성
flat = [item for pair in anp_multicollinear for item in [pair["feature1"], pair["feature2"]]]
col_counts = Counter(flat)

# 많이 중복된 변수 상위 출력
print("🔁 중복 등장한 변수 (빈도순):")
for col, count in col_counts.most_common():
    print(f"{col}: {count}회")

In [0]:
# 검증: 제거 대상 컬럼이 각 쌍에서 적어도 하나는 포함되었는지?
unresolved_pairs_np = []

for pair in anp_multicollinear:
    if pair["feature1"] not in drop_nonperiod_cols and pair["feature2"] not in drop_nonperiod_cols:
        unresolved_pairs_np.append(pair)

print(f"⚠️ 제거되지 않은 위험한 쌍 수: {len(unresolved_pairs_np)}")

In [0]:
print(f"제거 전 컬럼 수: {len(anp_df.columns)}")

In [0]:
# 제거 실행
anp_reduced_df = anp_df.drop(*drop_nonperiod_cols)

print(f"최종 컬럼 수 (클러스터링용): {len(anp_reduced_df.columns)}")

In [0]:
anp_reduced_df = anp_reduced_df.drop("이용금액_업종기준")
print(f"최종 컬럼 수 (클러스터링용): {len(anp_reduced_df.columns)}")

### 💡 데이터 추출

In [0]:
# spark.sql("DROP TABLE IF EXISTS database_pjt.3_amount_nonperiod")
spark.sql("DROP TABLE IF EXISTS database_03_cache.amount_nonperiod_df_fin")


In [0]:
### 데이터 베이스 사용 설정
anp_reduced_df = anp_reduced_df.cache()
spark.sql("USE database_pjt")
print("현재 데이터베이스를 'database_pjt'로 설정")

anp_reduced_df.write.mode("overwrite").saveAsTable("3_amount_nonperiod")
print("이용금액(기간 포함) 관련 테이블 생성 완료")

In [0]:
### 데이터 베이스 사용 설정
anp_reduced_df = anp_reduced_df.cache()
spark.sql("USE database_03_cache")
print("현재 데이터베이스를 'database_03_cache'로 설정")

anp_reduced_df.write.mode("overwrite").saveAsTable("amount_nonperiod_df_fin")
print("이용금액(기간 포함) 관련 테이블 생성 완료")

In [0]:
### 확인
df2 = spark.read.table("database_03_cache.amount_nonperiod_df_fin")

print(len(df2.columns))
df2.printSchema()